In [ ]:
! pip install selenium==3.141.0

In [ ]:
! pip install urllib3==1.26.11

In [ ]:
! pip install chardet==5.2.0

In [ ]:
! wget https://storage.googleapis.com/chrome-for-testing-public/133.0.6943.98/linux64/chromedriver-linux64.zip

In [ ]:
! unzip chromedriver-linux64.zip

In [ ]:
! sudo apt install -y libvulkan1

In [ ]:
! wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb

In [ ]:
! sudo dpkg -i google-chrome-stable_current_amd64.deb

In [ ]:
! ps -ef |grep chrome | awk '{print $2}' | xargs kill -9

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import os

download_dir = '/home/ubuntu/toxcast_collect/chemicals'
chromedriver_path = 'chromedriver-linux64/chromedriver'
chemicals_id_file = 'chemicals_id'

waiting_seconds = 10

# Set up Chrome options for headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration (recommended for headless)
chrome_options.add_argument("--no-sandbox")  # Prevents errors in certain environments (like Docker)

# Specify the download directory
prefs = {
    "download.default_directory": download_dir,  # Set download directory
    "download.prompt_for_download": False,        # Disable download prompt
    "directory_upgrade": True                     # Allow changing the download directory
}

chrome_options.add_experimental_option("prefs", prefs)

with open(chemicals_id_file, 'r') as f:
    lines = f.readlines()
    for i, l in enumerate(lines):
        chemical_name = l.replace('\n','')

        # Set up the WebDriver (Make sure you've downloaded the correct WebDriver for your browser)
        driver = webdriver.Chrome(executable_path=chromedriver_path, options=chrome_options)  # Provide your chromedriver path
        driver.set_window_size(1920, 1080)

        #driver.maximize_window()
        url = f'https://comptox.epa.gov/dashboard/chemical-lists/{chemical_name}'
        driver.get(url)  # Replace with the URL of the webpage you want to automate

        print(i, url)

        print('waiting for page loading')
        # Give the page some time to load
        time.sleep(waiting_seconds)  # You can adjust the sleep time depending on your network speed

        # Locate the <span> element with exact text "Export data"
        span_element = driver.find_element(By.XPATH, "//span[normalize-space()='Export Data']")

        driver.execute_script(f"window.scrollTo(0, {span_element.location['y']});")

        actions = ActionChains(driver)

        # Alternatively, using ActionChains (if needed)
        actions.move_by_offset(1615, 10).click()
        actions.move_by_offset(0, 80).click().perform()

        # waiting to be downloaded
        time.sleep(waiting_seconds)

        # Close the browser
        driver.quit()
